<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#methods" data-toc-modified-id="methods-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>methods</a></span></li></ul></li></ul></li><li><span><a href="#Planning" data-toc-modified-id="Planning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Planning</a></span></li><li><span><a href="#Scrape-candidate-IDs" data-toc-modified-id="Scrape-candidate-IDs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scrape candidate IDs</a></span><ul class="toc-item"><li><span><a href="#re-explore-the-scraping-html" data-toc-modified-id="re-explore-the-scraping-html-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>re-explore the scraping html</a></span></li><li><span><a href="#get-all-member-page-URLs" data-toc-modified-id="get-all-member-page-URLs-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>get all member page URLs</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#ID-scrape-complete,-save-objects" data-toc-modified-id="ID-scrape-complete,-save-objects-2.2.0.1"><span class="toc-item-num">2.2.0.1&nbsp;&nbsp;</span>ID scrape complete, save objects</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#make-api-calls" data-toc-modified-id="make-api-calls-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>make api calls</a></span><ul class="toc-item"><li><span><a href="#4-piece-API-call-methods" data-toc-modified-id="4-piece-API-call-methods-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>4-piece API call methods</a></span></li></ul></li><li><span><a href="#Parse-XML-into-nested-list-of-nested-dictionaries" data-toc-modified-id="Parse-XML-into-nested-list-of-nested-dictionaries-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Parse XML into nested list of nested dictionaries</a></span></li><li><span><a href="#data-ready-for-graphing" data-toc-modified-id="data-ready-for-graphing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>data ready for graphing</a></span></li></ul></div>

### imports

In [1]:
import pandas as pd
import selenium
import random
import time
import os
import io
import json 
import pickle
import requests
import xmltodict
import pprint
import json
import config


import xml.etree.ElementTree as ET
from io import StringIO
from random import randint
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

we'll be <s>scraping</s> requesting data from opensecrets.org (Center for Responsive Politics). specifically we're looking at each congressmember's campaign donations received from lobbying groups, organized by industry.

- data taken from Center for Responsive Politics @ opensecrets.org

### methods

In [54]:
def short_wait():
    time.sleep(random.randint(5,10))
    
# path = '/Users/mark/Documents/flatiron/projects/congress_bribes/graphing_congressional_lobbying/graphing_congress_lobbying_industries/'
def launch_driver(): # i should figure out a way to dynamically code the path so it launches on anyone's machine
    driver = webdriver.Chrome(executable_path='/Users/mark/Documents/flatiron/projects/congress_bribes/graphing_congressional_lobbying/graphing_congress_lobbying_industries/chromedriver', options=option)
    return driver # wouldn't that require keeping chromedriver executable in the project folder?

# i wonder if there's a jupyter extension to collapse these?

# ||||||||||| obtaining data
def get_attributes(element): # found a neat javascript to list all attributes of WebElement
    a = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', element)
    return a

def extract_links_from_page(): # extracts URLs for each member's page from list_of_members page
    odds = driver.find_elements_by_class_name('odd')
    evens = driver.find_elements_by_class_name('even')
    rows = odds + evens
    current_page_urls = []
    for row in rows: # extract link to congressmember's page
        a = row.find_element_by_tag_name('a')
        current_page_urls.append(a.get_attribute('href'))
    return current_page_urls # returns a list of URLS

def go_next_page():
    driver.find_element_by_id('DataTables_Table_0_next').click()
    
def update_url(url): # saves a couple of clicks per member, which is good. websites get angry if you click a lot
    tar = 'https://www.opensecrets.org/members-of-congress/industries?cid=N00007360&cycle=2020&recs=0&type=C'
    return tar[:59]+url[-13:]+tar[72:] # put unique member_id into updated url

def get_all_links():
    urls = []
    for i in range(11): # there's 11 pages. could code this better to a while, check for Next page = True
        page_urls = extract_links_from_page()
        urls += page_urls
        print(f'|||||scraped page {i+1}')
        print(f'# urls: {len(urls)}')
        short_wait()
        go_next_page()
        short_wait()
    print(f'storing {len(urls)} URLs in list')
    clean_urls = []
    for url in urls:
        clean_urls.append(update_url(url))
    return clean_urls

def get_member_ids(urls): # returns list of strings
    ids = []
    for url in urls:
        ids.append(url[63:72])
    return ids
        

def download_csv(): # downloads the relevant .csv to your default Downloads folder
    print('waiting 3-5 to load') # you'll have 535 new .csvs
    time.sleep(random.randint(3,5))
    holder = driver.find_element_by_class_name('table-title')
    holder.find_element_by_tag_name('a').click() # clicks download csv button
    
def extract_bribery_records(urls): # pass it your cleaned URL list
    print('|||beginning bribery extraction|||')
    for c, url in enumerate(urls): # loops through all 535 members, downloads
        print('waiting 4-7 before visiting...') # waiting is time consuming
        time.sleep(random.randint(4,7)) # but so is being IP banned from your target site
        driver.get(url)
        download_csv()
        print(f'downloaded {c} member\'s bribes')

        # API method
def get_responses(urls):
    responses = []
    for c, url in enumerate(urls):
        mem_id = url[63:72] # slice the candidate's unique ID from the links we gather earlier
        print(mem_id)
        req = 'https://www.opensecrets.org/api/?method=candIndustry&cid=N00007360&cycle=2018&apikey='+key
        new = req[:57]+mem_id+req[66:73]+'2020'+req[77:] # splice into requests URL
        r = requests.get(new)
        time.sleep(random.randint(1,2))
        if r.status_code!=200:
            print('___---___---___ Bad Request')
        responses.append(r)
        print(f'{c} requests made')
    print('|||done requesting|||')
    return responses
        
# ||||||||||| cleaning data
def read_all_csvs(path): # reads all csvs in working directory to list of dataframes
    dfs = []
    c=0 # this method.... i'm not using this anymore
    c+=1 # filename slicing, add name to each dataframe
    df = pd.read_csv(file) # file is the filename string
    df['name'] = file[15:file.index('N0')-1].replace(' ','_')
    dfs.append(df)
    print(f'read {c} dataframes. returning list...')
    return dfs
            
def pickle_obj(obj, filename): # designed with list of strings in mind, but i guess it works for other stuff? cool
    urls = open(filename,"w+")
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
def unpickle_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)
    
    
# |||||||||||||||||||||||||||||||||||| API XML cleaning

def write_resps_to_xml(resps): # pass the list of responses
    for c,resp in enumerate(resps):
        name = resp.text[1:4] + str(c) + '.xml' # arbitrary filename, xml extension
        with open(name, 'wb') as f: # we can extract the member's name from the XML later
            f.write(resp.content) # save the file to working directory
            print(f'wrote {c} resps to directory')
#         time.sleep(1)
            
def read_xmls_to_dict(): # reads all .XMLs in working directory
    members = {} # returns dictionary with nested member-dictionaries (holdin)
    directory = os.path.join("c:\\",path)
    for root,dirs,files in os.walk(directory):
        for file in files: # initially i despaired at the lack of JSON in the API's responses
            if file.endswith(".xml"): # but ElementTree is super helpful here
                mem_dict = {}
                tree = ET.parse(file)
                root = tree.getroot()
                mem_dict['name'] = root[0].attrib['cand_name'] # set identifier
                mem_dict['party'] = mem_dict['name'][-2]
                for industries_tag in root: # there's only 1
                    for indu in industries_tag: # however many different industries donated to this candidate
                        in_dict = {}
                        in_dict['code'] = indu.attrib['industry_code']
                        in_dict['name'] = indu.attrib['industry_name']
                        in_dict['indiv'] = indu.attrib['indivs']
                        in_dict['PAC'] = indu.attrib['pacs']
                        in_dict['total'] = indu.attrib['total']
                        mem_dict[in_dict['name']] = in_dict # nest industry into member
                members[mem_dict['name']] = mem_dict # nest member into holder dict
    if len(members)==535: print('we\'re good')
    return members 
    
    

# Planning

- for each member of congress, get:
    - name, party, list of industries
    - for each industry, get:
        - name, dollars donated, financial breakdown
        
we'll be using data from the Center for Responsive Politics (opensecrets.org).
- scrape IDs from webpage
- use IDs in api calls
- parse XML responses to list of nested dictionaries
- save list for graphing in next step

# Scrape candidate IDs
- their API has a getLegislators method
    - but that only returns the current congress's IDs
    - if we want past data, we can *gently* scrape IDs from the site and substitute the proper year in the call URL

In [3]:
pwd

'/Users/mark/Documents/flatiron/projects/congress_bribes/graphing_congressional_lobbying/graphing_congress_lobbying_industries'

In [50]:
# variables for driver launch
base_url = 'https://www.opensecrets.org/members-of-congress/members-list?cong_no=116&cycle=2020&sort=N'
option = webdriver.ChromeOptions() # you'll need the proper chrome drivers for Selenium
option.add_argument(' — incognito', )
# substitute your proper working directory
path = '/Users/mark/Documents/flatiron/projects/congress_bribes/graphing_congressional_lobbying/graphing_congress_lobbying_industries/'

In [51]:
# help(option.add_argument)

In [56]:
driver = launch_driver()

In [53]:
key = config.api_key
# you'll need to register on opensecrets.org to get an API key

## re-explore the scraping html

In [57]:
driver.get("https://www.opensecrets.org/members-of-congress/members-list?cong_no=116&cycle=2018&sort=N")


In [58]:
abcs = driver.find_elements_by_tag_name('p')

In [59]:
len(abcs)

35

In [61]:
type(abcs[0])

selenium.webdriver.remote.webelement.WebElement

## get all member page URLs

In [7]:
def get_member_ids(): # scrape candidate unique IDs from searchpage to use in API queries
    print('sleeping before accessing url. . .')
    time.sleep(random.randint(5,8))
    driver.get("https://www.opensecrets.org/members-of-congress/members-list?cong_no=116&cycle=2018&sort=N")
    chamber_select = Select(driver.find_element_by_name('Chamber')) # select chamber dropdown
    chambers = ['House', 'Senate'] # 430 house, #104 senate. use for filtering
    lst = [] # list of lists to return
    for chamber in chambers:
        chamber_select.select_by_visible_text(chamber) # handles the click part
        if chamber == 'House': max_links = 430
        else: max_links = 104
        mem_ids = []
        while len(mem_ids) < max_links:
            print('scraping new page')
            # on current page, find table of members
            table_outer = driver.find_element_by_id('DataTables_Table_0_wrapper')
            links = table_outer.find_elements_by_tag_name('a') # contains the hrefs to member pages
            links_lst = []
            try:
                for c, el in enumerate(links):
                    if c < 50:
                        links_lst.append(el)
                for c, link in enumerate(links_lst):
                    print(f'link {c}')
                    st = link.get_attribute('href')
                    print(st)
                    mem_id = st[60:69]
                    mem_ids.append(mem_id)
                    if c == 50: break
            except TypeError: print('you probably tried to slice a nonexistent string because you didn\'t make it stop before 30 on the last page')
            driver.find_element_by_id('DataTables_Table_0_next').click() # go next
            print('sleeping for 5...')
            time.sleep(random.randint(5,8)) # don't get b&
        print(len(mem_ids))
        lst.append(mem_ids)
    print(len(lst[0]),len(lst[1]))
    print('got all the links as a list. [ [house_links], [senate_links]]')
    return lst
            

In [54]:
id_list = get_member_ids()

sleeping before accessing url. . .
scraping new page
link 0
https://www.opensecrets.org/members-of-congress/summary?cid=N00036633&cycle=2020
link 1
https://www.opensecrets.org/members-of-congress/summary?cid=N00035451&cycle=2020
link 2
https://www.opensecrets.org/members-of-congress/summary?cid=N00003028&cycle=2020
link 3
https://www.opensecrets.org/members-of-congress/summary?cid=N00033997&cycle=2020
link 4
https://www.opensecrets.org/members-of-congress/summary?cid=N00033720&cycle=2020
link 5
https://www.opensecrets.org/members-of-congress/summary?cid=N00040989&cycle=2020
link 6
https://www.opensecrets.org/members-of-congress/summary?cid=N00031938&cycle=2020
link 7
https://www.opensecrets.org/members-of-congress/summary?cid=N00031177&cycle=2020
link 8
https://www.opensecrets.org/members-of-congress/summary?cid=N00042868&cycle=2020
link 9
https://www.opensecrets.org/members-of-congress/summary?cid=N00038285&cycle=2020
link 10
https://www.opensecrets.org/members-of-congress/summary?cid

sleeping for 5...
scraping new page
link 0
https://www.opensecrets.org/members-of-congress/summary?cid=N00000615&cycle=2020
link 1
https://www.opensecrets.org/members-of-congress/summary?cid=N00030693&cycle=2020
link 2
https://www.opensecrets.org/members-of-congress/summary?cid=N00040741&cycle=2020
link 3
https://www.opensecrets.org/members-of-congress/summary?cid=N00033449&cycle=2020
link 4
https://www.opensecrets.org/members-of-congress/summary?cid=N00030709&cycle=2020
link 5
https://www.opensecrets.org/members-of-congress/summary?cid=N00030957&cycle=2020
link 6
https://www.opensecrets.org/members-of-congress/summary?cid=N00031317&cycle=2020
link 7
https://www.opensecrets.org/members-of-congress/summary?cid=N00025337&cycle=2020
link 8
https://www.opensecrets.org/members-of-congress/summary?cid=N00036149&cycle=2020
link 9
https://www.opensecrets.org/members-of-congress/summary?cid=N00006023&cycle=2020
link 10
https://www.opensecrets.org/members-of-congress/summary?cid=N00001373&cycle=

sleeping for 5...
scraping new page
link 0
https://www.opensecrets.org/members-of-congress/summary?cid=N00035215&cycle=2020
link 1
https://www.opensecrets.org/members-of-congress/summary?cid=N00037003&cycle=2020
link 2
https://www.opensecrets.org/members-of-congress/summary?cid=N00034044&cycle=2020
link 3
https://www.opensecrets.org/members-of-congress/summary?cid=N00026427&cycle=2020
link 4
https://www.opensecrets.org/members-of-congress/summary?cid=N00033395&cycle=2020
link 5
https://www.opensecrets.org/members-of-congress/summary?cid=N00034453&cycle=2020
link 6
https://www.opensecrets.org/members-of-congress/summary?cid=N00041370&cycle=2020
link 7
https://www.opensecrets.org/members-of-congress/summary?cid=N00004403&cycle=2020
link 8
https://www.opensecrets.org/members-of-congress/summary?cid=N00001193&cycle=2020
link 9
https://www.opensecrets.org/members-of-congress/summary?cid=N00025237&cycle=2020
link 10
https://www.opensecrets.org/members-of-congress/summary?cid=N00030667&cycle=

sleeping for 5...
scraping new page
link 0
https://www.opensecrets.org/members-of-congress/summary?cid=N00034120&cycle=2020
link 1
https://www.opensecrets.org/members-of-congress/summary?cid=N00033591&cycle=2020
link 2
https://www.opensecrets.org/members-of-congress/summary?cid=N00004558&cycle=2020
link 3
https://www.opensecrets.org/members-of-congress/summary?cid=N00041134&cycle=2020
link 4
https://www.opensecrets.org/members-of-congress/summary?cid=N00013817&cycle=2020
link 5
https://www.opensecrets.org/members-of-congress/summary?cid=N00033549&cycle=2020
link 6
https://www.opensecrets.org/members-of-congress/summary?cid=N00040865&cycle=2020
link 7
https://www.opensecrets.org/members-of-congress/summary?cid=N00029662&cycle=2020
link 8
https://www.opensecrets.org/members-of-congress/summary?cid=N00042581&cycle=2020
link 9
https://www.opensecrets.org/members-of-congress/summary?cid=N00002260&cycle=2020
link 10
https://www.opensecrets.org/members-of-congress/summary?cid=N00030581&cycle=

https://www.opensecrets.org/members-of-congress/summary?cid=N00042164&cycle=2020
link 49
https://www.opensecrets.org/members-of-congress/summary?cid=N00007021&cycle=2020
sleeping for 5...
scraping new page
link 0
https://www.opensecrets.org/members-of-congress/summary?cid=N00033839&cycle=2020
link 1
https://www.opensecrets.org/members-of-congress/summary?cid=N00034349&cycle=2020
link 2
https://www.opensecrets.org/members-of-congress/summary?cid=N00001102&cycle=2020
link 3
https://www.opensecrets.org/members-of-congress/summary?cid=N00003813&cycle=2020
link 4
https://www.opensecrets.org/members-of-congress/summary?cid=N00033106&cycle=2020
link 5
https://www.opensecrets.org/members-of-congress/summary?cid=N00026368&cycle=2020
link 6
https://www.opensecrets.org/members-of-congress/summary?cid=N00007690&cycle=2020
link 7
https://www.opensecrets.org/members-of-congress/summary?cid=N00035311&cycle=2020
link 8
https://www.opensecrets.org/members-of-congress/summary?cid=N00031226&cycle=2020
li

sleeping for 5...
scraping new page
link 0
https://www.opensecrets.org/members-of-congress/summary?cid=N00033492&cycle=2020
link 1
https://www.opensecrets.org/members-of-congress/summary?cid=N00027533&cycle=2020
link 2
https://www.opensecrets.org/members-of-congress/summary?cid=N00003280&cycle=2020
link 3
https://www.opensecrets.org/members-of-congress/summary?cid=N00007724&cycle=2020
link 4
https://www.opensecrets.org/members-of-congress/summary?cid=N00030670&cycle=2020
link 5
None
you probably tried to slice a nonexistent string because you didn't make it stop before 30 on the last page
sleeping for 5...
105
430 105
got all the links as a list. [ [house_links], [senate_links]]


In [60]:
print(type(id_list), len(id_list), len(id_list[0]), len(id_list[1]), '\n', id_list[0][0])
# looking good

<class 'list'> 2 430 105 
 N00036633


In [13]:
filen = 'all_congress_ids'

In [5]:
pickle_obj(id_list, filen) # saved to working directory

NameError: name 'id_list' is not defined

#### ID scrape complete, save objects

In [14]:
t = unpickle_obj(filen)
id_list = t

In [15]:
# check for duplicates
all_ids = id_list[0]+id_list[1]
print(len(all_ids))
c = 0
for a in all_ids:
    for b in all_ids:
        if a == b: c+=1
print(c) # no duplicates in ID list

535
535


# make api calls

In [8]:
key = config.api_key
# the API has a 200-per-period limit on the candIndustry method we're using
# so until they get back to me, i'll use a piecemeal split-method approach to finish it in 3 periods

## 4-piece API call methods
- the 'one for everything' method i tried was messy and gave me duplicates, so let's do this cleanly

In [9]:
apk = config.api_key
def make_industry_request(cand_id):
    rst = "https://www.opensecrets.org/api/?method=candIndustry&cid="+cand_id+"&cycle=2018&apikey="+apk
    resp = requests.get(rst) # make the request
    if 'call limit' in resp.text:
        print('call limit reached! breaking to fix')
        return
    return resp
                
    
def get_first_200(responses, id_list, apk):
    print('first 200 of house')
    ids = id_list[0][:200] # designate first 200 of house
    for c, cand_id in enumerate(ids):
        resp = make_industry_request(cand_id)
        responses[0].append(resp)
        print(f'{c} / 200')
    print(len(responses[0]))
    return responses

def get_next_200(responses, id_list, apk):
    print('next 200 of house')
    ids = id_list[0][200:400] # designate next 200 of house
    for c, cand_id in enumerate(ids):
        resp = make_industry_request(cand_id)
        responses[0].append(resp)
        print(f'{c} / 200')
    print(len(responses[0]))
    return responses

def get_35_house(responses, id_list, apk):
    print('remaining 35 of house')
    ids = id_list[0][400:] # designate next 35 of house
    for c, cand_id in enumerate(ids):
        resp = make_industry_request(cand_id)
        responses[0].append(resp)
        print(f'{c} / {len(ids)}')
    print(len(responses[0]))
    return responses

def get_all_senate(responses, id_list, apk):
    print('all 105 of senate')
    ids = id_list[1] # designate senate
    for c, cand_id in enumerate(ids):
        resp = make_industry_request(cand_id)
        responses[1].append(resp)
        print(f'{c} / {len(ids)}')
    print(len(responses[1]))
    return responses
# precise but verbose

In [10]:
len(id_list[1])

NameError: name 'id_list' is not defined

In [29]:
# a case:switch method. kinda.
def populate_responses(resps, id_list, apk): # resps should be empty. [ [], [] ] structure
    if len(resps[0]) < 200: 
        resps = get_first_200(resps, id_list, apk)
    elif len(resps[0]) >= 200 and len(resps[0]) < 400:
        resps = get_next_200(resps, id_list, apk)
    elif len(resps[0]) >= 400 and len(resps[0]) < 430:
        resps = get_35_house(resps, id_list, apk)
    elif len(resps[1]) < 105:
        print('populating senate')
        resps = get_all_senate(resps, id_list, apk)
    print(f'house: {len(resps[0])} / 435   senate: {len(resps[1])} / 105')
    return resps # we'll have to run this 3 over 3 periods

In [12]:
resps = [[],[]]

In [13]:
# 1
print(f"start 1: {len(resps[0])}, {len(resps[1])}")
resps = populate_responses(resps, id_list, apk)
print(f"end: {len(resps[0])}, {len(resps[1])}")

start 1: 0, 0
first 200 of house
0 / 200
1 / 200
2 / 200
3 / 200
4 / 200
5 / 200
6 / 200
7 / 200
8 / 200
9 / 200
10 / 200
11 / 200
12 / 200
13 / 200
14 / 200
15 / 200
16 / 200
17 / 200
18 / 200
19 / 200
20 / 200
21 / 200
22 / 200
23 / 200
24 / 200
25 / 200
26 / 200
27 / 200
28 / 200
29 / 200
30 / 200
31 / 200
32 / 200
33 / 200
34 / 200
35 / 200
36 / 200
37 / 200
38 / 200
39 / 200
40 / 200
41 / 200
42 / 200
43 / 200
44 / 200
45 / 200
46 / 200
47 / 200
48 / 200
49 / 200
50 / 200
51 / 200
52 / 200
53 / 200
54 / 200
55 / 200
56 / 200
57 / 200
58 / 200
59 / 200
60 / 200
61 / 200
62 / 200
63 / 200
64 / 200
65 / 200
66 / 200
67 / 200
68 / 200
69 / 200
70 / 200
71 / 200
72 / 200
73 / 200
74 / 200
75 / 200
76 / 200
77 / 200
78 / 200
79 / 200
80 / 200
81 / 200
82 / 200
83 / 200
84 / 200
85 / 200
86 / 200
87 / 200
88 / 200
89 / 200
90 / 200
91 / 200
92 / 200
93 / 200
94 / 200
95 / 200
96 / 200
97 / 200
98 / 200
99 / 200
100 / 200
101 / 200
102 / 200
103 / 200
104 / 200
105 / 200
106 / 200
107 / 2

In [14]:
pickle_obj(resps, 'first_200')

In [42]:
len(resps[0])

200

In [43]:
# 2
print(f"start 2: {len(resps[0])}, {len(resps[1])}")
resps = populate_responses(resps, id_list, apk)
print(f"end: {len(resps[0])}, {len(resps[1])}")

start 2: 200, 0
next 200 of house
0 / 200
1 / 200
2 / 200
3 / 200
4 / 200
5 / 200
6 / 200
7 / 200
8 / 200
9 / 200
10 / 200
11 / 200
12 / 200
13 / 200
14 / 200
15 / 200
16 / 200
17 / 200
18 / 200
19 / 200
20 / 200
21 / 200
22 / 200
23 / 200
24 / 200
25 / 200
26 / 200
27 / 200
28 / 200
29 / 200
30 / 200
31 / 200
32 / 200
33 / 200
34 / 200
35 / 200
36 / 200
37 / 200
38 / 200
39 / 200
40 / 200
41 / 200
42 / 200
43 / 200
44 / 200
45 / 200
46 / 200
47 / 200
48 / 200
49 / 200
50 / 200
51 / 200
52 / 200
53 / 200
54 / 200
55 / 200
56 / 200
57 / 200
58 / 200
59 / 200
60 / 200
61 / 200
62 / 200
63 / 200
64 / 200
65 / 200
66 / 200
67 / 200
68 / 200
69 / 200
70 / 200
71 / 200
72 / 200
73 / 200
74 / 200
75 / 200
76 / 200
77 / 200
78 / 200
79 / 200
80 / 200
81 / 200
82 / 200
83 / 200
84 / 200
85 / 200
86 / 200
87 / 200
88 / 200
89 / 200
90 / 200
91 / 200
92 / 200
93 / 200
94 / 200
95 / 200
96 / 200
97 / 200
98 / 200
99 / 200
100 / 200
101 / 200
102 / 200
103 / 200
104 / 200
105 / 200
106 / 200
107 / 

In [55]:
len(resps[0])

430

In [21]:
resps = unpickle_obj('next_400')

In [22]:
len(resps[0])

397

In [23]:
type(resps[0][0])

dict

In [24]:
resps[0].append({'placeholder':'yes'}) # get the length to 400
resps[0].append({'placeholder':'yes'}) # so the populate() doesn't think it's still on day 2
resps[0].append({'placeholder':'yes'})
len(resps[0])

400

In [25]:
# 3
print(f"start 3: {len(resps[0])}, {len(resps[1])}")
resps = populate_responses(resps, id_list, apk)
print(f"end: {len(resps[0])}, {len(resps[1])}")
pickle_obj(resps, 'complete_430')
pickle_obj(resps, 'full_data_430')

start 3: 400, 0
remaining 35 of house
0 / 30
1 / 30
2 / 30
3 / 30
4 / 30
5 / 30
6 / 30
7 / 30
8 / 30
9 / 30
10 / 30
11 / 30
12 / 30
13 / 30
14 / 30
15 / 30
16 / 30
17 / 30
18 / 30
19 / 30
20 / 30
21 / 30
22 / 30
23 / 30
24 / 30
25 / 30
26 / 30
27 / 30
28 / 30
29 / 30
430
house: 430 / 435   senate: 0 / 105
end: 430, 0


In [30]:
# 4
print(f"start 3: {len(resps[0])}, {len(resps[1])}")
resps = populate_responses(resps, id_list, apk)
print(f"end: {len(resps[0])}, {len(resps[1])}")
pickle_obj(resps, 'complete_430')
pickle_obj(resps, 'full_data_430')

start 3: 432, 0
populating senate
all 105 of senate
0 / 105
1 / 105
2 / 105
3 / 105
4 / 105
5 / 105
6 / 105
7 / 105
8 / 105
9 / 105
10 / 105
11 / 105
12 / 105
13 / 105
14 / 105
15 / 105
16 / 105
17 / 105
18 / 105
19 / 105
20 / 105
21 / 105
22 / 105
23 / 105
24 / 105
25 / 105
26 / 105
27 / 105
28 / 105
29 / 105
30 / 105
31 / 105
32 / 105
33 / 105
34 / 105
35 / 105
36 / 105
37 / 105
38 / 105
39 / 105
40 / 105
41 / 105
42 / 105
43 / 105
44 / 105
45 / 105
46 / 105
47 / 105
48 / 105
49 / 105
50 / 105
51 / 105
52 / 105
53 / 105
54 / 105
55 / 105
56 / 105
57 / 105
58 / 105
59 / 105
60 / 105
61 / 105
62 / 105
63 / 105
64 / 105
65 / 105
66 / 105
67 / 105
68 / 105
69 / 105
70 / 105
71 / 105
72 / 105
73 / 105
74 / 105
75 / 105
76 / 105
77 / 105
78 / 105
79 / 105
80 / 105
81 / 105
82 / 105
83 / 105
84 / 105
85 / 105
86 / 105
87 / 105
88 / 105
89 / 105
90 / 105
91 / 105
92 / 105
93 / 105
94 / 105
95 / 105
96 / 105
97 / 105
98 / 105
99 / 105
100 / 105
101 / 105
102 / 105
103 / 105
104 / 105
105
hous

In [31]:
pickle_obj(resps, 'full_data')

In [90]:
# fortunately i've got the data saved locally so we can graph it

In [31]:
resps = unpickle_obj('first_200')

In [27]:
resps_backup = resps

# Parse XML into nested list of nested dictionaries
- it's a lot cleaner than it sounds

- same [ [house], [chamber] ] structure
    - { name, party, id, industries },
        - industries: { oil:3600, pharma:12000, ... }

In [33]:
def parse_xml(resp): # parses member's xml to dictionary
    d = {}
    ec = 0
    try:
        root = ET.fromstring(resp.text)
        for child in (root.iter('*')):
            cd = child.attrib # child dictionary
            if child.tag == 'industries': # identification
                d['name'] = cd['cand_name']
                d['party'] = d['name'][-2:-1]
                d['id'] = cd['cid']
            elif child.tag == 'industry': # parse bribe
                bribe = {}
                bribe['code'] = cd['industry_code']
                bribe['name'] = cd['industry_name']
                bribe['indiv'] = cd['indivs']
                bribe['pac'] = cd['pacs']
                bribe['total'] = cd['total']
                d[bribe['name']] = bribe # key is industry name, value is dict
            # d should have identifying attributes + several dictionaries for each bribe
    except:
        ec +=1
    print(f'{ec} errors')
    return d

def parse_responses(resps): # returns list of member dicts (with nested dictionaries for each)
    out = []
    for chamber in resps:
        ch = []
        for c, member in enumerate(chamber):
            mem_dict = parse_xml(member)
            ch.append(mem_dict)
            print(f'{c}')
        out.append(ch)
    print(len(out[0]), len(out[1]))
    return out
            
        

In [34]:
mems = parse_responses(resps)

1 errors
0
1 errors
1
1 errors
2
1 errors
3
1 errors
4
1 errors
5
1 errors
6
1 errors
7
1 errors
8
1 errors
9
1 errors
10
1 errors
11
1 errors
12
1 errors
13
1 errors
14
1 errors
15
1 errors
16
1 errors
17
1 errors
18
1 errors
19
1 errors
20
1 errors
21
1 errors
22
1 errors
23
1 errors
24
1 errors
25
1 errors
26
1 errors
27
1 errors
28
1 errors
29
1 errors
30
1 errors
31
1 errors
32
1 errors
33
1 errors
34
1 errors
35
1 errors
36
1 errors
37
1 errors
38
1 errors
39
1 errors
40
1 errors
41
1 errors
42
1 errors
43
1 errors
44
1 errors
45
1 errors
46
1 errors
47
1 errors
48
1 errors
49
1 errors
50
1 errors
51
1 errors
52
1 errors
53
1 errors
54
1 errors
55
1 errors
56
1 errors
57
1 errors
58
1 errors
59
1 errors
60
1 errors
61
1 errors
62
1 errors
63
1 errors
64
1 errors
65
1 errors
66
1 errors
67
1 errors
68
1 errors
69
1 errors
70
1 errors
71
1 errors
72
1 errors
73
1 errors
74
1 errors
75
1 errors
76
1 errors
77
1 errors
78
1 errors
79
1 errors
80
1 errors
81
1 errors
82
1 errors
83
1 

In [35]:
mems_backup = mems

In [36]:
len(mems[0])

432

In [30]:
mems[0][194]['name']

'Marcy Kaptur (D)'

In [40]:
mems = mems_backup

In [42]:
print(len(mems[0]), len(mems[1]))

232 104


In [37]:

def clean_nulls(mems):
    print(len(mems[0]))
    c=0
    for chamber in mems:
        for n, mem in enumerate(chamber):
            print(n)
            if not mem: # if dict is empty. probably an id mismatch on my end
                c+=1 # i might have used 2016 IDs for the 2018 congress. big oops
                chamber.remove(mem)
                continue
            a = mem['name']
    print(f'{c} nulls removed')
    print(f"{len(mems[0])}")
    
clean_nulls(mems)

432
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62

In [39]:
c=0 # check for duplicates
for chamber in mems:
    for c, mem in enumerate(chamber):
        a = mem['name']
        for mem in chamber: # double loop to check doubles
            b = mem['name']
            if a==b: c+=1
c -= len(mems[0])
print(f'{len(mems[0])} mems,  {c} matches' ) # we want no matches

KeyError: 'name'

In [51]:
mems

[[{'name': 'Ralph Abraham (R)',
   'party': 'R',
   'id': 'N00036633',
   'Health Professionals': {'code': 'H01',
    'name': 'Health Professionals',
    'indiv': '32400',
    'pac': '55500',
    'total': '87900'},
   'Crop Production & Basic Processing': {'code': 'A01',
    'name': 'Crop Production & Basic Processing',
    'indiv': '30100',
    'pac': '52250',
    'total': '82350'},
   'Agricultural Services/Products': {'code': 'A07',
    'name': 'Agricultural Services/Products',
    'indiv': '750',
    'pac': '37500',
    'total': '38250'},
   'General Contractors': {'code': 'C01',
    'name': 'General Contractors',
    'indiv': '29100',
    'pac': '4500',
    'total': '33600'},
   'Air Transport': {'code': 'M01',
    'name': 'Air Transport',
    'indiv': '0',
    'pac': '30500',
    'total': '30500'},
   'Leadership PACs': {'code': 'Q03',
    'name': 'Leadership PACs',
    'indiv': '0',
    'pac': '29750',
    'total': '29750'},
   'Defense Aerospace': {'code': 'D01',
    'name': 'D

In [52]:
pickle_obj(mems, 'next_400')
print('data saved!')

data saved!


# data ready for graphing
- continued in 'graphing' notebook